In [1]:
### Imports
import string
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import ImageColorGenerator

In [2]:
pd.set_option("display.max_rows", None)

# Preprocessing of Text for Analysis Purposes

## Import the new dataframe
(that has artist names removed from transcripts)

In [3]:
df = pd.read_json('../raw_data/df_all_clean .json')

In [4]:
len(df)

555

## Cleaning Functions

In [7]:
### Specific functions to our data - Scraps from the Loft and BBC

# remove notes (Bo Burnham Only)
def clean_bo(text):
    txt = text
    for note in '♫♪':
        txt = txt.replace(note, '')
    return txt

def remove_music(text):
    text = re.sub('♪.*?♪', '', text) # remove ♪ stuff that looks like this ♪
    text = re.sub('♫.*?♫', '', text) # remove ♫ stuff that looks like this ♫
    return text

def remove_bracketed(text):
    text = re.sub('\[.*?\]', '', text) # remove [stuff that looks like this]
    text = re.sub('\(.*?\)', '', text) # remove (stuff that looks like this)
    return text

def remove_speaker_tags(text):
    text = re.sub('\s[\w-]+( \w+)?:\s', ' ', text) # remove Word: or Word word: with a newline or space before
    return text

def remove_info(text):
    text = re.sub('subtitle(s)? by .*', '', str(text)) # remove subtile(s) by xxxx
    text = re.sub('(a)? netflix (original )?(comedy )?(special ?)?', '', text) # remove A Netflix Original Comedy Special
    text = re.sub('(this )?(programme )?(contains )?(very |some )?strong language( |\.)', '', text) # remove strong language
    text = re.sub('adult humou?r( |\.?)?', '', text) # remove adult humour
    text = re.sub('(original )?air date', '', text) # remove air date
    return text

In [8]:
# general functions for text pre-processing
def remove_punc(text, chars):
    txt = text
    for punc in chars:
        txt = txt.replace(punc, '')
    return txt

def remove_num(text):
    return ''.join(char for char in text if not char.isdigit())

def remove_stopw(text, word_list):
    word_tokens = word_tokenize(text)
    return ' '.join(w for w in word_tokens if not w in word_list)

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split(' ') if len(lemmatizer.lemmatize(word))>2)

In [9]:
lemmatizer_dict = {'got': 'get',
                  'gon': 'go',
                  'said': 'say',
                   'saying': 'say',
                  'fucking': 'fuck',
                  'went': 'go',
                  'finding': 'find',
                  'getting': 'get'}

def manual_lemmatizer(text):
    for k, v in lemmatizer_dict.items():
        text = text.replace(k, v)
    return text

## Modifying & applying removal lists 

In [10]:
clean_df = df.copy()

### Remove everything in Brackets, Music notes

In [11]:
clean_df = df.copy()

## Step 1: Remove everything in Brackets, Music notes
clean_df['full_transcript_clean'] = clean_df['full_transcript'].apply(remove_bracketed)

### clean Bo before removing music
clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'] = clean_df[
    'full_transcript_clean'][clean_df['artist']=='Bo Burnham'].apply(clean_bo)

clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_music)

/var/folders/s8/jf61hd8n2cj411r4h4mdpyf00000gn/T/ipykernel_88261/137205631.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'] = clean_df[


### Lowercase, remove useless regex matches
Including specific scraps/BBC format

In [12]:
# lowercase all the words
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].str.lower()

# remove speaker tags and info regexes
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_info)
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_speaker_tags)

### Remove numbers and stopwords + common comedy words, remove punctuation


In [13]:
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_num)

In [14]:
### additional words to remove from the scripts
words_to_remove = ['thank', 'cheering', 'recorded', 'applause', 'laughter', 'laughing', 'murmuring', 'chatter',
                       'aired', 'filmed', 'ladies', 'gentlemen', 'welcome', 'stage', 'transcript', 'netflix',
                  'apollo', 'like', 'goodnight', 'mutter', 'noo', 'nuh', 'oof', 'maan', 'fuck', 'cause', 'okay', 
                   'hey', 'also', 'someone', 'somebody', 'everybody', 'also', 'part' , 'sometimes', 'maybe', 
                   'three', 'second', 'everything', 'minute', 'name', 'kind', 'point', 'yeah', 'hello', 'one', 
                   'two', 'four', 'five', 'six', 'seven', 'eight', 'nine']
                    
                    # 'know'? 'go'? 'fuck'?
    
                    # haven't left 'i'm' etc. as those should be cleaned up
                    # by a mixture of stopwords, punctuation removeal, lemmatizing and minimum length

stopwords_plus = words_to_remove + stopwords.words('english')

clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_stopw, args=(stopwords_plus,))

In [15]:
punc_added = string.punctuation + '“”‘’…♪♫¶'

clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_punc, args=(punc_added,))

### Lemmatize

In [16]:
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(lemmatize).apply(manual_lemmatizer)

In [17]:
clean_df.head()

,full_transcript,artist,show_name,year,source,full_transcript_clean
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,man let right guy much take seat guy get jacke...
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,yes yes yes well lovely going apologise state ...
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,please host tonight london london live amazing...
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,please host tonight know hair lost bet british...
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,going start say something probably never heard...


## LDA model to see topics

In [ ]:
new_sw = ['whine', 'hnn', 'malla', 'letta', 'namoo', 'getta', 'nama', 'mana', 'chk',
         'manoo', 'hadda', 'ama', 'carlin']
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_stopw, args=(new_sw,))

In [ ]:
vectorizer = TfidfVectorizer().fit(clean_df['full_transcript_clean'])
data_vectorized = vectorizer.transform(clean_df['full_transcript_clean'])
lda_model = LatentDirichletAllocation(n_components=2).fit(data_vectorized)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

## Word frequency

In [ ]:
token_df = clean_df['full_transcript_clean'].apply(tokenize).astype(str)
token_df.head(20)

In [ ]:
pd.Series([y for x in token_df.values.flatten() for y in x.split()]).value_counts().head(50)

## Bag of words

In [ ]:
def dict_bow(df):
    keys = df.sum().index
    values = df.sum().values
    bow_dict = dict(zip(keys, values))
    return bow_dict

In [ ]:
# first bow

t = clean_df['full_transcript_clean'].astype(str)

vectorizer = CountVectorizer(min_df = 0.3, max_df = 0.8, ngram_range=(1,1))
X = vectorizer.fit_transform(t)
bow_df1 = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())
len(bow_df1.columns)

In [ ]:
dict_1 = dict_bow(bow_df1)

In [ ]:
# second bow

t = clean_df['full_transcript_clean'].astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, ngram_range=(1,1))
X = vectorizer.fit_transform(t)
bow_df2 = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())

dict_2 = dict_bow(bow_df2)

In [ ]:
# third bow

t = clean_df['full_transcript_clean'].astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, max_features=50, ngram_range=(1,2))
X = vectorizer.fit_transform(t)
bow_df3 = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())

dict_3 = dict_bow(bow_df3)

In [ ]:
# fourth bow

t = clean_df['full_transcript_clean'].astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.7, max_features=50, ngram_range=(1,2))
X = vectorizer.fit_transform(t)
dict_4 = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names()))

## Word cloud with frequencies

In [18]:
emoji_mask = np.array(Image.open('../raw_data/emoji.png'))

In [19]:
def make_word_cloud(wc, bow_dict):
    word_cloud = wc.generate_from_frequencies(bow_dict)
    return word_cloud

In [22]:
# Define a function to plot word cloud
def plot_cloud(wordcloud, mask):
    # Set figure size
    plt.figure(figsize=(30, 20))
    # Display image
    image_colors = ImageColorGenerator(mask)
    plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear") 
    # No axis details
    plt.axis("off");

In [21]:
wc = WordCloud(width=3000, height = 2000, background_color='white',
                       random_state=1, collocations=False, stopwords = STOPWORDS, mask=emoji_mask)

In [23]:
def dict_bow(df):
    keys = df.sum().index
    values = df.sum().values
    bow_dict = dict(zip(keys, values))
    return bow_dict

In [ ]:
def plot_some_cloud(index, mask):
    t = clean_df['full_transcript_clean'][index]
    if type(t)!=str:
        t = t.astype(str)
        vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, ngram_range=(1,2), max_features=80)
    else:
        vectorizer = CountVectorizer(max_features=80)
    X = vectorizer.fit_transform(t)
    temp_dict = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out()))
    word_cloud = make_word_cloud(wc, dict_5)
    plot_cloud(word_cloud, mask)

In [ ]:
plot_some_cloud(emoji_mask)

In [ ]:
# fifith bow

temp_series.clean_df['full_transcript_clean'].copy()
t = temp_series.astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, ngram_range=(1,2), max_features=80)

X = vectorizer.fit_transform(t)
dict_5 = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out()))

word_cloud = make_word_cloud(wc, dict_5)
plot_cloud(word_cloud, emoji_mask)

In [ ]:
df[df['artist'] == 'George Carlin'].index

In [ ]:
df[df['artist'] == 'Mae Martin'].index[0]

In [ ]:
temp_series.iloc[360]

In [ ]:
t = pd.Series(temp_series.iloc[360])
vectorizer = CountVectorizer(ngram_range=(1,2), max_features=40)

X = vectorizer.fit_transform(t)
dict_6 = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out()))

word_cloud = make_word_cloud(wc, dict_6)
plot_cloud(word_cloud)

In [ ]:
EI = temp_series.iloc[163:168]
t = EI.astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, ngram_range=(1,2), max_features=80)

X = vectorizer.fit_transform(t)
dict_6 = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out()))

word_cloud = make_word_cloud(wc, dict_6)
plot_cloud(word_cloud)

In [ ]:
carlin_script = temp_series.iloc[190:210]

t = carlin_script.astype(str)

vectorizer = CountVectorizer(min_df = 0.4, max_df = 0.8, ngram_range=(1,2), max_features=80)

X = vectorizer.fit_transform(t)
dict_6 = dict_bow(pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out()))

word_cloud = make_word_cloud(wc, dict_6)
plot_cloud(word_cloud)

# ----- **note to self: start from here**-----

In [ ]:
full_transcripts = ' '.join(clean_df['full_transcript_clean'])

In [ ]:
len(full_transcripts)

In [ ]:
word_cloud = WordCloud(width=3000, height = 2000,
                       random_state=1, colormap='Pastel1',
                       collocations=False, stopwords = STOPWORDS).generate(full_transcripts)

In [ ]:
plot_cloud(word_cloud)